# Analisis de informacion de Grupo Interact

##### Conexion a base de datos MySQL para obtener informacion almacenada

In [2]:
import pandas as pd
from sqlalchemy import create_engine

localhost = 'localhost'
user = 'root'
password = 'Mysql12345'
database = 'reportehoras'

# Configurar conexión y probarla

In [3]:
def cargar_tabla(nombre_tabla, con, chunk_size=1000):
    """
    Carga tabla en un DF usando chunks para optimizar la lectura.
    :param nombre_tabla: # Nombre de la tabla
    :param con: # Conexion a la base de datos
    :param chunk_size: # tamano de los pedazos
    :return: Object tipo DataFrame
    """
    return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)

try:
    con = create_engine(f'mysql+pymysql://{user}:{password}@{localhost}/{database}')
    if con is not None:      
        
        tablas = ["clasificaciones", "clientes", "datos_act", "empleados", "modalidades"]
        dfs = {tabla: cargar_tabla(tabla, con) for tabla in tablas}
        
        df_clasificaciones = dfs['clasificaciones']
        df_clientes = dfs['clientes']
        df_datos_act = dfs['datos_act']
        df_empleados = dfs['empleados']
        df_modalidades = dfs['modalidades']
                
except RuntimeError as e:
    print(f'❌ Contraseña incorrecta')
    con = None
except Exception as err:
    print(f"❌ Error de conexión: {err}")
    con = None
finally:
    if con is not None:
        con.dispose()

C:\Users\variz\AppData\Local\Temp\ipykernel_10844\1221055656.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(pd.read_sql(f'SELECT * FROM {nombre_tabla}', con=con, chunksize=chunk_size), ignore_index=True)


##### Realizar merge para obtener df con variables categoricas de llaves foraneas

In [4]:
df_datos_act.head()

,clave_act,numero,clave_cli,clave_mod,clave_cla,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion,bloqueado,tipo,horas,fecha_capt,hora_capt,usuario,baja,tiempo_tot,tiempo_desc
0,1,9011,18,1,22,2024-05-01,07:00,10:00,0.0,0.0,*Reunión para planeación del día con los compa...,1.0,A,None,2024-10-08,10:59:26,9030,0,3.0,3.0
1,2,9011,25,1,45,2024-05-01,10:00,10:30,0.0,0.0,*Soporte a A. Chávez en pólizas entregadas de ...,1.0,A,None,2024-10-08,10:59:26,9030,0,0.5,0.5
2,3,9034,18,1,22,2024-05-01,08:00,09:00,0.0,0.0,capacitación 5s,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
3,4,9034,18,1,22,2024-05-01,09:00,10:00,0.0,0.0,Capacitacion proyecto de reporte de horas,1.0,A,None,2024-06-11,08:55:45,9034,0,1.0,1.0
4,5,9010,18,1,3,2024-05-01,07:00,16:00,0.0,1.0,"Revisión de temas de red nueva oficina, ubicac...",1.0,A,None,2024-06-11,08:57:08,9034,0,9.0,8.0


In [5]:
# Se revisa parte del df_clasificaciones ya que no permite merge por index 'tipo'
df_clasificaciones.columns
df_clasificaciones.columns = df_clasificaciones.columns.str.strip()
df_clasificaciones.head()

,clave_cla,descripcion,tipo,fecha_capt,hora_capt,usuario,baja
0,1,Soporte,S,2024-04-15,14:34:00,1111,2
1,2,Proyectos,P,2024-04-15,14:35:00,1111,2
2,3,Actividades Administrativas,S,2024-05-01,11:19:40,9010,0
3,4,Actividades Contables,S,2024-05-01,11:19:55,9010,0
4,5,Actualización del Sistema,S,2024-05-01,11:20:15,9010,0


In [6]:
df_concentrado = pd.merge(df_datos_act, df_clientes[['clave_cli', 'nombre']], how='left', on='clave_cli', suffixes=('_datos','_cli'))
df_concentrado = df_concentrado.merge(df_modalidades[['clave_mod','descripcion']], how='left', on='clave_mod', suffixes=('_total','_modalidades'))
df_concentrado = df_concentrado.merge(df_clasificaciones[['clave_cla','descripcion','tipo']], how='left', on='clave_cla', suffixes=('_total', '_clasificaciones'))
df_concentrado = df_concentrado.merge(df_empleados[['numero', 'nombre']], how='left', on='numero', suffixes=('_total', '_empleados'))
df_concentrado.drop(columns=['clave_cli', 'clave_mod', 'clave_cla', 'numero', 'usuario', 'bloqueado', 'horas'], inplace=True)

df_concentrado = df_concentrado[['clave_act', 'nombre_empleados', 'nombre_total', 'descripcion_modalidades', 'descripcion', 'fecha_act', 'hora_ini', 'hora_fin', 'tiempo_t', 'tiempo_p', 'descripcion_total', 'tipo_clasificaciones', 'fecha_capt', 'hora_capt', 'tiempo_tot', 'tiempo_desc']]

df_concentrado.tail()


,clave_act,nombre_empleados,nombre_total,descripcion_modalidades,descripcion,fecha_act,hora_ini,hora_fin,tiempo_t,tiempo_p,descripcion_total,tipo_clasificaciones,fecha_capt,hora_capt,tiempo_tot,tiempo_desc
6157,6165,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-09,07:00,17:10,0.0,1.0,Avance en creación de servicio y conexión a FTP,P,2025-01-14,17:17:21,10.17,9.17
6158,6166,JORGE ARIZA RAMIREZ,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-14,07:00,11:00,0.0,0.0,Revisión y corrección de detalles en servicio ...,P,2025-01-14,17:18:07,4.00,4.00
6159,6167,JORGE ARIZA RAMIREZ,GRUPO INTERACT SA DE CV,Presencial Oficina,Juntas y Reuniones,2025-01-14,11:00,12:30,0.0,0.0,Revisión de avances y correcciones con Nataly....,S,2025-01-14,17:19:13,1.50,1.50
6160,6168,JORGE ARIZA RAMIREZ,THYSSENKRUP COMPONENTS GUANAJUATO,Presencial Oficina,Implementación de sistemas nuevo cliente,2025-01-14,12:30,14:00,0.0,0.0,Avance en creación de servicio y conexión a ca...,P,2025-01-14,17:20:48,1.50,1.50
6161,6169,JORGE ARIZA RAMIREZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,Presencial Oficina,Generación de credenciales,2025-01-14,14:00,17:00,0.0,1.0,Impresión de credenciales.,S,2025-01-14,17:21:49,3.00,2.00


In [7]:
# from pyecharts.charts import Bar
# from pyecharts import options as opts
# # Built-in theme types can be found in pyecharts.globals.ThemeType
# from pyecharts.globals import ThemeType
# 
# bar = (
#     Bar(init_opts=opts.InitOpts(theme=ThemeType.MACARONS))
#     .add_xaxis(["shirt", "cardigan", "chiffon", "trousers", "heels", "socks"])
#     .add_yaxis("Merchant A", [5, 20, 36, 10, 75, 90])
#     .add_yaxis("Merchant B", [15, 6, 45, 20, 35, 66])
#     .add_yaxis("Merchant C", [5, 20, 36, 10, 75, 90])
#     .set_global_opts(title_opts=opts.TitleOpts(title="Main Title", subtitle="Sub Title"))
# )
# 
# bar.render("render.html")

In [8]:
# Convertir atributos a tipos de objeto datetime para trabajar con fechas

df_concentrado['fecha_capt'] = pd.to_datetime(df_concentrado['fecha_capt'])
df_concentrado['fecha_act'] = pd.to_datetime(df_concentrado['fecha_act'])
df_concentrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   clave_act                6162 non-null   int64         
 1   nombre_empleados         6162 non-null   object        
 2   nombre_total             6162 non-null   object        
 3   descripcion_modalidades  6162 non-null   object        
 4   descripcion              6162 non-null   object        
 5   fecha_act                6162 non-null   datetime64[ns]
 6   hora_ini                 6162 non-null   object        
 7   hora_fin                 6162 non-null   object        
 8   tiempo_t                 6162 non-null   float64       
 9   tiempo_p                 6162 non-null   float64       
 10  descripcion_total        6162 non-null   object        
 11  tipo_clasificaciones     6162 non-null   object        
 12  fecha_capt               6162 non-

In [19]:
df_concentrado['anio'] = df_concentrado['fecha_act'].dt.year
df_concentrado['mes'] = df_concentrado['fecha_act'].dt.month
resumen = df_concentrado.groupby(['anio', 'mes', 'nombre_empleados', 'nombre_total'])['tiempo_desc'].sum().reset_index()
resumen = resumen.sort_values(by=['anio', 'mes', 'nombre_empleados', 'nombre_total'])

# Mostrar el resultado
resumen

,anio,mes,nombre_empleados,nombre_total,tiempo_desc
0,2024,5,ANA CERVANTES PIEDRAS,ERGON ASFALTOS MEXICO S DE RL DE CV,12.58
1,2024,5,ANA CERVANTES PIEDRAS,ERGON MEXICO S DE RL DE CV,0.98
2,2024,5,ANA CERVANTES PIEDRAS,GRUPO INTERACT SA DE CV,180.99
3,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,GRUPO INTERACT SA DE CV,86.42
4,2024,5,ARITZI JANET HERNÁNDEZ HERRERA,OXIQUIMICA SA DE CV,69.55
...,...,...,...,...,...
497,2025,1,JORGE ARIZA RAMIREZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,2.00
498,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,GRUPO INTERACT SA DE CV,10.00
499,2025,1,JOSE ALFONSO HERNANDEZ ESPINOZA,THYSSENKRUP COMPONENTS GUANAJUATO,54.67
500,2025,1,MIRIAM FABIAN LUNA,GRUPO INTERACT SA DE CV,66.59


In [20]:
resumen.query("anio == 2024 and mes == 9 and nombre_empleados == 'NATALY ALCANTARA HERNANDEZ'")

,anio,mes,nombre_empleados,nombre_total,tiempo_desc
305,2024,9,NATALY ALCANTARA HERNANDEZ,CONDOMINIO CORPORATIVO ANGELOPOLIS AC,1.00
306,2024,9,NATALY ALCANTARA HERNANDEZ,DIVISION CEMMTHOME,6.25
307,2024,9,NATALY ALCANTARA HERNANDEZ,EUWE EUGEN WEXLER DE MEXICO SA DE CV,6.25
308,2024,9,NATALY ALCANTARA HERNANDEZ,GRAMMER AUTOMOTIVE PUEBLA SA DE CV,2.17
309,2024,9,NATALY ALCANTARA HERNANDEZ,GRUPO INTERACT SA DE CV,107.92
310,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,16.33
311,2024,9,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,41.50


In [24]:
resumen["porcentaje_total"] = resumen.groupby(["anio", "mes", "nombre_empleados"])["tiempo_desc"].transform(lambda x: (x / x.sum())*100)
resumen.query("anio == 2024 and mes == 8 and nombre_empleados == 'NATALY ALCANTARA HERNANDEZ'")


,anio,mes,nombre_empleados,nombre_total,tiempo_desc,porcentaje_total
250,2024,8,NATALY ALCANTARA HERNANDEZ,EUWE EUGEN WEXLER DE MEXICO SA DE CV,7.33,3.603559
251,2024,8,NATALY ALCANTARA HERNANDEZ,GRUPO INTERACT SA DE CV,165.42,81.323435
252,2024,8,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP AUTOMOTIVE SYSTEMS DE MEXICO SA D...,1.00,0.491618
253,2024,8,NATALY ALCANTARA HERNANDEZ,THYSSENKRUPP PRESTA DE MEXICO SA DE CV,29.66,14.581387
